# Import the necessary modules needed for analysis in python

In [45]:
import pandas as pd
import sklearn as sk

- Read in the file which is shared as a csv

In [54]:
df = pd.read_csv("..\datasets\pbp-2020.csv")

df.head()
df.columns

Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'OffenseTeam',
       'DefenseTeam', 'Down', 'ToGo', 'YardLine', 'SeriesFirstDown',
       'NextScore', 'Description', 'TeamWin', 'SeasonYear', 'Yards',
       'Formation', 'PlayType', 'IsRush', 'IsPass', 'IsIncomplete',
       'IsTouchdown', 'PassType', 'IsSack', 'IsChallenge',
       'IsChallengeReversed', 'Challenger', 'IsMeasurement', 'IsInterception',
       'IsFumble', 'IsPenalty', 'IsTwoPointConversion',
       'IsTwoPointConversionSuccessful', 'RushDirection', 'YardLineFixed',
       'YardLineDirection', 'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay',
       'PenaltyType', 'PenaltyYards'],
      dtype='object')

- Filter the dataset to create subsets for KC and SF

In [47]:
kc_df = df[(df['OffenseTeam'] == 'KC') | (df['DefenseTeam'] == 'KC')]

kc_df.head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020091000,9/10/2020,4,7,15,HOU,KC,0,0,98,...,1,0,NaN,2,OPP,0,NaN,0,NaN,0
1,2020091000,9/10/2020,4,14,15,KC,HOU,1,10,83,...,0,0,LEFT GUARD,17,OPP,0,NaN,0,NaN,0
3,2020091000,9/10/2020,2,0,4,KC,HOU,1,10,89,...,0,0,NaN,11,OPP,0,NaN,0,NaN,0
4,2020091000,9/10/2020,2,0,9,KC,HOU,1,10,84,...,0,0,NaN,16,OPP,1,HOU,1,ILLEGAL CONTACT,5
5,2020091000,9/10/2020,2,0,16,KC,HOU,1,10,58,...,0,0,NaN,42,OPP,1,HOU,0,HORSE COLLAR TACKLE,15


In [48]:
sf_df = df[(df['OffenseTeam'] == 'SF') | (df['DefenseTeam'] == 'SF')]

sf_df.head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
190,2020091311,9/13/2020,4,0,37,SF,ARI,4,5,84,...,0,0,NaN,16,OPP,0,NaN,0,NaN,0
192,2020091311,9/13/2020,4,0,47,SF,ARI,3,5,84,...,0,0,NaN,16,OPP,0,NaN,0,NaN,0
193,2020091311,9/13/2020,4,1,14,SF,ARI,2,10,79,...,0,0,NaN,21,OPP,0,NaN,0,NaN,0
194,2020091311,9/13/2020,4,1,21,SF,ARI,1,10,79,...,0,0,NaN,21,OPP,0,NaN,0,NaN,0
195,2020091311,9/13/2020,4,1,43,SF,ARI,2,8,70,...,0,0,NaN,30,OPP,0,NaN,0,NaN,0


- Aggregate the total number of yards per season for each team

In [85]:
kc_tot_yards = kc_df.groupby(['OffenseTeam', 'DefenseTeam'])['Yards'].sum().reset_index()

kc_tot_yards.head()

,OffenseTeam,DefenseTeam,Yards
0,ATL,KC,387
1,BAL,KC,237
2,BUF,KC,210
3,CAR,KC,435
4,DEN,KC,816


In [83]:
sf_tot_yards = sf_df.groupby(['OffenseTeam', 'DefenseTeam'])['Yards'].sum().reset_index()

sf_tot_yards.head()

,OffenseTeam,DefenseTeam,Yards
0,ARI,SF,780
1,BUF,SF,461
2,DAL,SF,291
3,GB,SF,423
4,LA,SF,662


- Get the number of yard given up in penalties by game per season

In [51]:
kc_penalty_yards = kc_df[kc_df['PenaltyTeam'] == 'KC'].groupby(['OffenseTeam', 'DefenseTeam'])['PenaltyYards'].sum().reset_index()

kc_penalty_yards.head()

,OffenseTeam,DefenseTeam,PenaltyYards
0,ATL,KC,16
1,BAL,KC,29
2,BUF,KC,53
3,CAR,KC,45
4,DEN,KC,44


In [84]:
sf_penalty_yards = sf_df[sf_df['PenaltyTeam'] == 'SF'].groupby(['OffenseTeam', 'DefenseTeam'])['PenaltyYards'].sum().reset_index()

sf_penalty_yards.head()

,OffenseTeam,DefenseTeam,PenaltyYards
0,ARI,SF,65
1,BUF,SF,29
2,DAL,SF,0
3,GB,SF,13
4,LA,SF,42


- Subtract all of the penalty yards from the total yards for the season for each team

In [53]:
yards_with_penalties_by_team = kc_df.groupby(['OffenseTeam', 'DefenseTeam'])[['Yards', 'PenaltyYards']].sum().reset_index()

yards_with_penalties_by_team['TotalYardsIncludingPenalties'] = yards_with_penalties_by_team['Yards'] - yards_with_penalties_by_team['PenaltyYards']


yards_with_penalties_by_team.head()

,OffenseTeam,DefenseTeam,Yards,PenaltyYards,TotalYardsIncludingPenalties
0,ATL,KC,387,51,336
1,BAL,KC,237,49,188
2,BUF,KC,210,62,148
3,CAR,KC,435,90,345
4,DEN,KC,816,82,734


# Find the number of sacks allowed in the entire season, then determine what quarter they occur the most in.

- Starting with KC first

In [64]:
# kc_df_sacks = kc_df[kc_df['IsSack'] == 1]

kc_df_sacks = kc_df[kc_df['OffenseTeam'] == 'KC'].groupby(['DefenseTeam'])[['IsSack']].sum().reset_index()

kc_df_sacks

,DefenseTeam,IsSack
0,ATL,1
1,BAL,1
2,BUF,1
3,CAR,2
4,DEN,5
5,HOU,1
6,LAC,3
7,LV,3
8,MIA,3
9,NE,1


In [65]:
kc_total_sacks = kc_df[kc_df['OffenseTeam'] == 'KC'].groupby(['OffenseTeam'])[['IsSack']].sum().reset_index()

kc_total_sacks

,OffenseTeam,IsSack
0,KC,27


- Now do the same for SF

In [76]:
sf_df_sacks = (sf_df[sf_df['OffenseTeam'] == 'SF']).groupby(['DefenseTeam'])[['IsSack']].sum().reset_index()

sf_df_sacks

,DefenseTeam,IsSack
0,ARI,6
1,BUF,0
2,DAL,2
3,GB,2
4,LA,3
5,MIA,6
6,NE,1
7,NO,2
8,NYG,2
9,NYJ,3


In [79]:
sf_total_sacks = sf_df[sf_df['OffenseTeam'] == 'SF'].groupby(['OffenseTeam'])[['IsSack']].sum().reset_index()

sf_total_sacks

,OffenseTeam,IsSack
0,SF,42


So, we can see that SF received a total of 42 sacks during this season, and KC received a total of 27. 

This not only shows that SF is more prone to sacks, but let's find out when they are more prone to sacks.

In [82]:
kc_quarter_sacks = (kc_df[kc_df['OffenseTeam'] == 'KC']).groupby(['Quarter'])[['IsSack']].sum().reset_index()

kc_quarter_sacks

,Quarter,IsSack
0,1,6
1,2,10
2,3,5
3,4,6
4,5,0


In [81]:
sf_quarter_sacks = (sf_df[sf_df['OffenseTeam'] == 'SF']).groupby(['Quarter'])[['IsSack']].sum().reset_index()

sf_quarter_sacks

,Quarter,IsSack
0,1,11
1,2,11
2,3,13
3,4,7


- After a quick analysis, you can see that KC more prone to sacks in the 3rd quarter for this season.

- SF seems to have pretty vulnerable in the first half, with the highest frequency of sacks occurring in the 3rd quarter.

# Now we want to 